- Total de funcionários da empresa ok
- Total de funcionários por gênero ok 
- Frequência de funcionários por idade; ok
- Frequência de funcionários por faixa de idade;
- Distribuição de funcionários por nível hierárquico
- Distribuição de funcionários por Escolaridade
- Disitribuição de funcionários por Área da empresa
- Salário médio por nível hierárquico
- Salário médio por Escolaridade
- Quantidade de funcionários por mês de admissão (MoM) e percentual total
- Qual mês obteve o maior índice de crescimento de admissões?
- Quantidade de funcionários demitidos por mês (MoM) e percentual total
- Quantidade de funcionários por Tipo de contrato
- Relação de áreas vs tipo de contrato
- Somatório dos benefícios
- Quantidade de funcionários em férias, afastados, licenças

In [1]:
-- Visualizando a base de dados
SELECT TOP(10) * FROM baseRH

(10 rows affected)

Total execution time: 00:00:00.077

Nome,Gênero,Data_Nascimento,Idade,Nível,Escolaridade,Área,Salário,Data_Admissão,Data_Demissão,Contrato,Motivo_Demissão,PCD,Alimentação,Saúde,Odont,Transporte,Creche,Combustível,Benefícios,Mês_admissão,Mês_demissão,Status,Faixa_Idade,Tempo_dias,Faixa_dias
Kylie Rame,Masculino,1996-03-17,28,Supervisor,Ensino Médio,Financeiro,7000,2019-01-01,NULL,CLT,NULL,NULL,1200,1200,1200,1200,NULL,500,5300,1,-,NULL,24-34,364,270-364
Dwayne Betun,Masculino,1988-02-28,36,Gerente,Pós-graduação,Recursos Humanos,15000,2019-01-02,NULL,CLT,NULL,NULL,1200,1200,1200,1200,NULL,500,5300,1,-,Férias,35-44,363,270-364
Chrisse Oxbrough,Masculino,1988-09-22,36,Gerente,Ensino Superior,Marketing,15000,2019-01-02,NULL,CLT,NULL,NULL,1200,1200,1200,1200,NULL,500,5300,1,-,NULL,35-44,363,270-364
Kimmy Fardy,Feminino,1988-03-14,36,Gerente,Pós-graduação,Produção,15000,2019-01-02,NULL,CLT,NULL,NULL,1200,1200,1200,1200,NULL,500,5300,1,-,NULL,35-44,363,270-364
Mata Steynor,Masculino,1994-06-28,30,Analista Sr,Ensino Técnico,Produção,5000,2019-01-02,NULL,CLT,NULL,NULL,600,600,600,600,NULL,500,2900,1,-,NULL,24-34,363,270-364
Rivi Ofen,Feminino,1981-10-19,43,Supervisor,Ensino Superior,Financeiro,7000,2019-01-03,NULL,CLT,NULL,NULL,1200,1200,1200,1200,NULL,500,5300,1,-,NULL,35-44,362,270-364
Ross Tutchell,Masculino,1991-05-14,33,Assistente,Ensino Médio,Produção,2500,2019-01-03,NULL,CLT,NULL,NULL,600,600,600,600,500,NULL,2900,1,-,NULL,24-34,362,270-364
Lisa De Antoni,Feminino,1961-04-28,63,Estagiário,Ensino Médio,Administração,1200,2019-01-03,NULL,Estágio,NULL,NULL,200,200,200,200,NULL,NULL,800,1,-,NULL,55-64,362,270-364
Bethena Josephs,Feminino,1978-08-08,46,Analista Jr,Ensino Técnico,TI,3500,2019-01-03,NULL,CLT,NULL,NULL,600,600,600,600,NULL,NULL,2400,1,-,NULL,45-54,362,270-364
Kellia Koppel,Feminino,1970-07-19,54,Assistente,Ensino Superior,Recursos Humanos,2500,2019-01-03,NULL,CLT,NULL,NULL,600,600,600,600,NULL,NULL,2400,1,-,NULL,45-54,362,270-364


In [24]:
-- Total de funcionários na empresa ao longo do tempo, ativos e inativos
DECLARE @funcionarios_ativos INT = (SELECT COUNT(*) FROM baseRH WHERE Data_Demissão IS NOT NULL)
DECLARE @funcionarios_inativos INT = (SELECT COUNT(*) FROM baseRH WHERE Data_Demissão IS NULL)

SELECT  
    COUNT(*) AS Total_histórico,
    @funcionarios_ativos AS Ativos,
    FORMAT(1.0 * @funcionarios_ativos/COUNT(*), '0.00%') AS Percentual_Ativos,
    @funcionarios_inativos AS Inativos,
    FORMAT(1.0 * @funcionarios_inativos/COUNT(*), '0.00%') AS Percentual_Inativos
FROM baseRH

(1 row affected)

Total execution time: 00:00:00.014

Total_histórico,Ativos,Percentual_Ativos,Inativos,Percentual_Inativos
1000,71,"7,10%",929,"92,90%"


In [36]:
CREATE VIEW baseRH_ativos AS
(
    SELECT  
        *
    FROM baseRH
    WHERE Data_Demissão IS NULL
)

Commands completed successfully.

Total execution time: 00:00:00.009

In [39]:
-- Total de funcionários por genero
DECLARE @Total_funcionarios INT = (SELECT COUNT(*) FROM baseRH_ativos)

SELECT
    Gênero,
    COUNT(*) AS Total,
    FORMAT(1.0 * COUNT(*)/@Total_funcionarios, '0.00%') AS Percentual
FROM baseRH_ativos
GROUP BY Gênero

(2 rows affected)

Total execution time: 00:00:00.012

Gênero,Total,Percentual
Masculino,468,"50,38%"
Feminino,461,"49,62%"


In [41]:
-- Frequência de funcionários por idade
DECLARE @Total_funcionarios INT = (SELECT COUNT(*) FROM baseRH_ativos)

SELECT
    Faixa_Idade,
    COUNT(*) AS Total,
    FORMAT(1.0 * COUNT(*)/@Total_funcionarios, '0.00%') AS Percentual
FROM baseRH_ativos
GROUP BY Faixa_Idade

(4 rows affected)

Total execution time: 00:00:00.015

Faixa_Idade,Total,Percentual
24-34,235,"25,30%"
35-44,237,"25,51%"
45-54,232,"24,97%"
55-64,225,"24,22%"


50,38% dos funcionários são do gênero Masculino e 49,62% feminino.

In [2]:
-- Total de funcionários por mês de admissão e MoM
WITH CTE AS
(
SELECT
    MONTH(Data_Admissão) AS 'Mês_admissão',
    COUNT(*) AS 'Total'
FROM baseRH
GROUP BY MONTH(Data_Admissão)
)

SELECT 
    Mês_admissão,
    Total,
    FORMAT(1.0 * Total/SUM(Total)OVER(), '0.00%') AS 'Percentual_total',
    FORMAT(1.0 * (Total - LAG(Total,1)OVER(ORDER BY Mês_admissão ASC))/Total, '0.00%') AS 'MoM'
FROM CTE
ORDER BY Mês_admissão ASC

(12 rows affected)

Total execution time: 00:00:00.368

Mês_admissão,Total,Percentual_total,MoM
1,103,"10,30%",NULL
2,68,"6,80%","-51,47%"
3,87,"8,70%","21,84%"
4,80,"8,00%","-8,75%"
5,90,"9,00%","11,11%"
6,91,"9,10%","1,10%"
7,89,"8,90%","-2,25%"
8,90,"9,00%","1,11%"
9,72,"7,20%","-25,00%"
10,82,"8,20%","12,20%"


O mês com a maior quantidade de contratações foi em Janeiro, no total 103 (10,30%) admitidos.

O mês com o maior crescimento de contratações foi em em Março com 21,84% em relação ao mês anterior.